In [77]:
import os
os.environ["OMP_NUM_THREADS"] = "32"

In [78]:
from graph_tool.all import *
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.covariance import LedoitWolf, OAS
#import matplotlib.pyplot as py
#import seaborn as sns
import statsmodels.api as sm
from multipy.fdr import qvalue
from multipy.fdr import lsu

In [79]:
from trim_networks import *
from fit_sbm import *

In [80]:
### Input expression data
folder = "/Genomics/ayroleslab2/lamaya/bigProject/eQTLcatalog/modularity/matrices/"
gene_expr_body = pd.read_table(folder + "VOOMCounts_CPM1_body_ctrl_939ind_covfree_Aug3121.txt")
gene_expr_head = pd.read_table(folder + "VOOMCounts_CPM1_head_ctrl_940ind_covfree_Aug3121.txt")
gene_expr_dict = {"head": gene_expr_head.T, "body": gene_expr_body.T}

In [81]:
folder = "../data/output/SBM/graphs/"
g_body = load_graph(folder + "VOOMCounts_CPM1_body_ctrl_939ind_covfree_Aug3121.xml.gz")
g_head = load_graph(folder + "VOOMCounts_CPM1_head_ctrl_940ind_covfree_Aug3121.xml.gz")

g_dict = {"head": g_head, "body": g_body}

In [82]:
def getGeneNetworkStats(g, tissue):
    genes = g.vertex_properties["genes"]
    corr = g.edge_properties["spearman"]
    block_df = pd.DataFrame(columns=('Gene', 'Tissue', "Average_Spearman",
                                     'Degree_thr_0.1', 'WeightedDegree_thr_0.1',
                                     'Degree_thr_0.2', 'WeightedDegree_thr_0.2',
                                     'Degree_thr_0.3', 'WeightedDegree_thr_0.3',
                                     'Degree_thr_0.4', 'WeightedDegree_thr_0.4',
                                     'Degree_thr_0.5', 'WeightedDegree_thr_0.5',
                                     'Degree_fdr_5e-2' , 'WeightedDegree_fdr_5e-2',
                                     'Degree_fdr_1e-2' , 'WeightedDegree_fdr_1e-2',
                                     'Degree_fdr_1e-3' , 'WeightedDegree_fdr_1e-3',
                                     'Degree_fdr_1e-4' , 'WeightedDegree_fdr_1e-4',
                                     'Degree_fdr_1e-5' , 'WeightedDegree_fdr_1e-5',
                                     'Degree_fdr_1e-6' , 'WeightedDegree_fdr_1e-6'))

    tv_01 = filterByEdge(g, "spearman", 0.1, False)
    tv_02 = filterByEdge(g, "spearman", 0.2, False)
    tv_03 = filterByEdge(g, "spearman", 0.3, False)
    tv_04 = filterByEdge(g, "spearman", 0.4, False)
    tv_05 = filterByEdge(g, "spearman", 0.5, False)

    tv_fdr5e2 = filterByFDR(g, 5e-2, False)
    tv_fdr1e2 = filterByFDR(g, 1e-2, False)
    tv_fdr1e3 = filterByFDR(g, 1e-3, False)
    tv_fdr1e4 = filterByFDR(g, 1e-4, False)
    tv_fdr1e5 = filterByFDR(g, 1e-5, False)
    tv_fdr1e6 = filterByFDR(g, 1e-6, False)

    for v in g.vertex_index:
        line = [genes[v]]
        line.append(tissue)
        line.append(np.mean(np.abs(g.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_01.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_01.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_02.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_02.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_03.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_03.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_04.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_04.get_all_edges(v, [corr] )[:,2])))
        
        line.append(tv_05.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_05.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_fdr5e2.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_fdr5e2.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_fdr1e2.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_fdr1e2.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_fdr1e3.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_fdr1e3.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_fdr1e4.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_fdr1e4.get_all_edges(v, [corr] )[:,2])))
        
        line.append(tv_fdr1e5.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_fdr1e5.get_all_edges(v, [corr] )[:,2])))

        line.append(tv_fdr1e6.get_total_degrees([v])[0])
        line.append(np.mean(np.abs(tv_fdr1e6.get_all_edges(v, [corr] )[:,2])))

        block_df.loc[v] = line
    return block_df


In [83]:
gene_stats = {"body": getGeneNetworkStats(g_dict["body"], "body"), 
              "head": getGeneNetworkStats(g_dict["head"], "head") }

/Genomics/grid/users/damelo/.conda/envs/gt/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Genomics/grid/users/damelo/.conda/envs/gt/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [84]:
gene_stats["body"].append(gene_stats["head"])

,Gene,Tissue,Average_Spearman,Degree_thr_0.1,WeightedDegree_thr_0.1,Degree_thr_0.2,WeightedDegree_thr_0.2,Degree_thr_0.3,WeightedDegree_thr_0.3,Degree_thr_0.4,...,Degree_fdr_1e-2,WeightedDegree_fdr_1e-2,Degree_fdr_1e-3,WeightedDegree_fdr_1e-3,Degree_fdr_1e-4,WeightedDegree_fdr_1e-4,Degree_fdr_1e-5,WeightedDegree_fdr_1e-5,Degree_fdr_1e-6,WeightedDegree_fdr_1e-6
0,FBgn0031081,body,0.038366,270,0.113440,0,NaN,0,NaN,0,...,158,0.120611,26,0.143284,6,0.157670,1,0.168032,0,NaN
1,FBgn0031080,body,0.034086,142,0.114631,0,NaN,0,NaN,0,...,93,0.120563,18,0.139415,1,0.161738,0,NaN,0,NaN
2,FBgn0053217,body,0.027842,26,0.110019,0,NaN,0,NaN,0,...,15,0.113821,0,NaN,0,NaN,0,NaN,0,NaN
3,FBgn0052350,body,0.057266,1374,0.134153,49,0.217163,0,NaN,0,...,1141,0.140415,604,0.160620,336,0.177346,212,0.188960,120,0.201009
4,FBgn0024733,body,0.108547,3554,0.193872,1127,0.305479,342,0.461753,152,...,3320,0.200239,2563,0.224517,2033,0.246402,1676,0.265091,1404,0.282785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8872,FBgn0031309,head,0.030923,77,0.112013,0,NaN,0,NaN,0,...,30,0.124679,7,0.145454,2,0.164880,1,0.169586,0,NaN
8873,FBgn0031305,head,0.093991,2858,0.203305,1041,0.313529,442,0.411247,228,...,2565,0.214554,2016,0.240131,1665,0.260940,1420,0.278580,1231,0.294615
8874,FBgn0016926,head,0.053447,1227,0.134456,68,0.223093,0,NaN,0,...,934,0.143815,489,0.165292,279,0.183672,179,0.197259,117,0.209819
8875,FBgn0031299,head,0.031718,81,0.110465,0,NaN,0,NaN,0,...,24,0.123319,6,0.137336,0,NaN,0,NaN,0,NaN


In [85]:
gene_stats["body"].append(gene_stats["head"]).to_csv("../data/output/connectivity_stats_VOOMCounts_CPM1_headbody_ctrl_onlygenesinmainchr_covfree_Jul21.21.csv")

In [88]:
g_body = load_graph("../data/output/SBM/graphs/body_fdrLevel-1e-05_genes-3253_density-0.052.xml.gz")
g_head = load_graph("../data/output/SBM/graphs/head_fdrLevel-1e-05_genes-2777_density-0.023.xml.gz")

In [89]:
saveGeneTable(g_body, gene_expr_dict["body"], "body_table_wcgna_fdrLevel-1e-05.csv")

,FBgn0052350,FBgn0024733,FBgn0040372,FBgn0023537,FBgn0025640,FBgn0025635,FBgn0003575,FBgn0025638,FBgn0040383,FBgn0025621,...,FBgn0031377,FBgn0031360,FBgn0000579,FBgn0021906,FBgn0053126,FBgn0040723,FBgn0031313,FBgn0031305,FBgn0016926,FBgn0003310
108_A1,3.631374,12.837670,5.930541,7.417961,6.875142,5.112396,4.783182,7.952128,1.944521,4.836404,...,3.719237,5.634736,10.12944,8.669278,6.024389,6.382073,6.196365,2.883034,7.506171,5.675858
108_A11,4.533379,12.464833,6.575951,7.511536,5.095305,3.541294,4.802949,7.849514,3.005688,4.959199,...,5.505085,4.622778,10.32676,8.978416,6.468690,5.835634,5.615747,4.303585,7.483977,5.823808
108_A10,5.009452,12.043078,6.629539,6.298133,5.807446,4.667535,4.632493,7.883801,2.174565,4.298692,...,5.590217,4.470297,10.14280,8.366664,6.157096,6.362035,5.849679,4.597894,6.662226,5.001076
108_A7,5.194316,12.661575,5.769386,7.413696,5.631165,5.402705,4.237419,7.842671,4.925776,1.673321,...,3.750532,4.752270,10.41529,8.988652,6.236003,5.372080,5.701734,4.705742,7.450288,5.465879
108_A6,5.788053,12.536069,6.323843,7.069937,6.331945,5.394605,4.773309,7.848009,4.622032,4.700375,...,4.926975,5.095858,10.29835,8.617367,6.989220,7.312027,5.964033,4.073098,6.664993,5.655253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25_H8,4.713132,12.692008,5.737855,6.680325,6.647149,4.743285,4.814011,8.273734,4.588864,4.313809,...,3.499890,5.469305,10.26138,8.956402,6.676572,4.585973,5.907569,3.442014,7.272948,5.030097
25_H7,5.060531,12.461594,6.007094,6.711075,5.666485,4.893159,4.613515,8.062296,4.083897,4.491054,...,4.867379,4.913135,10.21620,8.766338,6.115881,5.887077,5.724326,4.648121,7.065589,5.469359
25_H6,4.914074,12.850248,5.563093,6.697851,5.442950,3.468940,4.975275,7.903197,4.338956,3.586287,...,4.310078,5.251274,10.19327,9.005600,7.001674,7.053956,5.616400,4.078457,7.510513,4.789762
25_H5,3.534570,12.609712,5.790243,7.176412,5.087064,4.571719,3.896724,7.999810,4.040774,3.374791,...,4.954121,5.354383,10.46144,9.169589,6.617927,6.608975,6.530754,5.093771,6.924951,5.792803


In [90]:
saveGeneTable(g_head, gene_expr_dict["head"], "head_table_wcgna_fdrLevel-1e-05.csv")

,FBgn0031081,FBgn0024733,FBgn0023537,FBgn0000108,FBgn0040383,FBgn0040382,FBgn0015288,FBgn0002579,FBgn0026879,FBgn0040349,...,FBgn0000579,FBgn0031359,FBgn0021906,FBgn0041097,FBgn0028481,FBgn0031306,FBgn0031312,FBgn0031313,FBgn0031305,FBgn0016926
106_B2,5.406171,12.159023,7.970223,9.103658,4.504749,8.910613,6.497881,12.074908,7.205130,5.818190,...,10.689870,6.250278,9.180181,4.528078,5.632289,7.911705,5.504739,7.256368,5.346317,7.725940
106_B7,5.264851,12.690366,7.888633,9.122704,5.771554,8.583128,5.818541,12.319365,7.285467,5.272072,...,10.163640,6.669228,8.684299,4.081323,5.426528,7.499327,4.418743,5.500916,5.720488,7.457484
106_B9,4.904944,11.897288,8.231778,9.339864,5.340754,8.769746,6.075431,12.024808,7.198762,5.814005,...,10.464990,6.428836,8.976875,4.467649,6.393772,8.001274,5.285779,6.344157,5.031428,8.094754
106_C12,5.565698,12.106231,7.819119,9.295989,5.556996,8.948759,5.947590,11.931441,7.441724,6.593676,...,10.448060,5.892828,8.296710,4.896429,6.044093,7.569231,5.329498,6.249799,6.333087,8.002430
106_A10,5.512900,11.664736,7.509090,9.560822,4.180259,8.803803,5.295312,11.472042,6.666197,3.997167,...,10.109830,7.310346,8.605877,4.350555,6.205850,8.071977,4.174026,4.904105,5.503860,8.149036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91_H4,4.816727,11.837820,7.547785,9.421305,4.806540,9.094638,5.497679,11.897016,7.038511,4.466682,...,10.358310,6.765676,8.777466,4.629994,6.079691,7.879473,4.648002,7.452780,5.638809,7.898581
91_H5,5.150994,12.217719,7.591914,9.327846,5.009814,8.606035,5.255452,12.034101,6.837308,4.900604,...,10.243170,6.631315,8.320069,3.703436,5.636800,7.527500,4.756286,6.463174,5.397940,7.858602
91_H7,4.696298,12.212852,7.062760,8.795936,5.203751,8.157917,5.307487,12.231420,7.564537,6.075576,...,9.780165,6.828104,8.983467,4.327952,5.706286,5.100653,5.591077,5.845839,-0.108135,7.678070
91_H8,4.858214,11.561459,7.298211,9.523435,4.707979,8.613049,5.218480,11.586817,6.717524,2.646312,...,10.199270,7.078562,8.391101,4.838711,5.755454,7.706873,4.680207,7.035322,4.440940,8.032992
